In [1]:
import magma as m
m.set_mantle_target("coreir")
import mantle

This notebook demonstrates using native Python functions to construct circuits.

In [2]:
def clb(a, b, c, d):
    return (a & b) | (~c & d)

T = m.UInt(16)
class Combinational(m.Circuit):
    name = "Combinational"
    IO = ["a", m.In(T), "b", m.In(T), "c", m.Out(T)]
    @classmethod
    def definition(io):
        m.wire(clb(io.a, io.b, io.a, io.b), io.c)

In [3]:
from magma.simulator import PythonSimulator
from magma.bit_vector import BitVector

simulator = PythonSimulator(Combinational)
a = BitVector(148, num_bits=16)
b = BitVector(41, num_bits=16)
simulator.set_value(Combinational.a, a)
simulator.set_value(Combinational.b, b)
simulator.evaluate()
assert BitVector(simulator.get_value(Combinational.c)) == clb(a, b, a, b)
print("Success!")

Success!


In [4]:
m.compile("build/Combinational", Combinational, output="coreir")
%cat build/Combinational.json

{"top":"global.Combinational",
"namespaces":{
  "global":{
    "modules":{
      "Combinational":{
        "type":["Record",[
          ["a",["Array",16,"BitIn"]],
          ["b",["Array",16,"BitIn"]],
          ["c",["Array",16,"Bit"]]
        ]],
        "instances":{
          "inst0":{
            "genref":"coreir.and",
            "genargs":{"width":["Int",16]}
          },
          "inst1":{
            "genref":"coreir.not",
            "genargs":{"width":["Int",16]}
          },
          "inst2":{
            "genref":"coreir.and",
            "genargs":{"width":["Int",16]}
          },
          "inst3":{
            "genref":"coreir.or",
            "genargs":{"width":["Int",16]}
          }
        },
        "connections":[
          ["self.a","inst0.in0"],
          ["self.b","inst0.in1"],
          ["inst3.in0","inst0.out"],
          ["self.a","inst1.in"],
          ["inst2.in0","inst1.out"],
          ["self.b","inst2.in1"],
          ["inst3.in1","inst2.out"],
      